In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
# 1. Data Preprocessing (MNIST dataset)
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))])

# 2. Data Preprocessing (EMNIST Letters dataset for A-Z)
# EMNIST Letters provides 26 classes (A-Z).
NUM_CLASSES = 26 
# Define the transformation pipeline
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,)) ])

In [3]:
#For the larger MNIST dataset
train_dataset = datasets.MNIST(root="./data", train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

#For the larger EMNIST dataset
# *** This is the key update, using split='letters' ***
train_dataset = datasets.EMNIST(root="./data", split='letters', train=True, transform=transform, download=True)
test_dataset = datasets.EMNIST(root="./data", split='letters', train=False, transform=transform, download=True)

# Set Data Loaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [4]:
# 2. Define CNN Model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        # Convolution layers
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1) 
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1) 
        self.pool = nn.MaxPool2d(2, 2)  
        # Fully connected layers
        self.fc1 = nn.Linear(32 * 7 * 7, 128)  # Flatten -> Dense
        self.fc2 = nn.Linear(128, 10)  # 10 classes for digits (0–9)
    def forward(self, x):
        # Convolution + ReLU + Pooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        # Flatten
        x = x.view(-1, 32 * 7 * 7)

        # Fully Connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # no activation here (softmax applied in loss)
        return x

In [5]:
# 2. Define CNN Model
class CNNModel(nn.Module):
    def __init__(self, num_classes=26): 
        super(CNNModel, self).__init__()
        # Convolution layers (kept same for feature extraction)
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1) 
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1) 
        self.pool = nn.MaxPool2d(2, 2)  
        # Fully connected layers
        self.fc1 = nn.Linear(32 * 7 * 7, 128)  
        self.fc2 = nn.Linear(128, num_classes) 

    def forward(self, x):
        # Convolution + ReLU + Pooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        # Flatten
        x = x.view(-1, 32 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  
        return x

In [6]:
# 3. Initialize Model, Loss, Optimizer MNIST
model = CNNModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [7]:
#Training loop
data_description = "Handwritten digit images"
print(f"Starting training on {'MNIST'.upper()} ({data_description}) for {3} epochs")

for epoch in range(3):
    model.train()  
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        labels = labels - 1  # shift labels from 1-26 → 0-25
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{3}, Average Loss: {avg_loss:.4f}")

Starting training on MNIST (Handwritten digit images) for 3 epochs
Epoch 1/3, Average Loss: 0.5811
Epoch 2/3, Average Loss: 0.2614
Epoch 3/3, Average Loss: 0.2156


In [9]:
# 5. testing 
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        labels = labels - 1  # Map 1-26 to 0-25
        
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on test data: {accuracy:.2f}%")

Accuracy on test data: 92.70%
